In [ ]:
import urllib
import requests
from IPython.core.display import HTML
def css_styling():
    styles = requests.get("https://raw.githubusercontent.com/saadtony/NumericalMethods/master/styles/custom.css")
    return HTML(styles.text)
css_styling()

# 引言
对于不可压缩CFD的控制方程，通过有限体积法对其离散后，得到了相关变量的代数方程组。那么如何求解代数方程组，就至关重要。通常矩阵$Ax=b$的求解，有两大类方法，一类是直接解法，一类是间接解法又称迭代解法。矩阵求解是个非常大的话题，本门课程主要分析讲解迭代解法中的常见方法，Jacobi，Gauss-Seidel，CG，MG等，旋转这些方法也是OpenFOAM软件中常见的算法，具有代表性。

# 直接解法
直接解法在不可压缩CFD中使用较少，主要原因是有限体积法离散后所得到的动量和压力修正线性方程组都是稀疏的，采用直接解法较慢，故涉及较少，有兴趣的同学可以了解一下。直接解法可以得到矩阵的精确解，这是其优点。缺点是内存开销大，计算速度慢等。常见的直接解法，高斯消元、LU分解和TDMA等。提一句，TDMA算法速度还是很快的，但是实用性较弱。

# 迭代解法
由于直接解法的内存开销大、计算速度慢等弱点，所以只能选择迭代解法。以一个例题开始我们的迭代法之旅。考虑如下所示线性方程组

$$
\tag{1}
\begin{array}{ccc}
5x+2y+2z=9\\
2x-6y+3z=-1\\
x+2y+7z=10
\end{array}
$$

将方程组$1$进行调整，得到如下形式
$$\tag{2a}x^n=\dfrac{9-2y^o-2z^o}{5}$$

$$\tag{2b}y^n=\dfrac{1+2x^o+3z^o}{6}$$

$$\tag{2c}z^n=\dfrac{10-x^o-2y^o}{7}$$

迭代解法的思路
1. 猜测矩阵$(x,y,z)'$的初始解，例如$(0,0,0)'$。
2. 通过第一步给定的$y,z$计算出新的$x^n$
3. 通过第一步给定的$x,z$计算出新的$y^n$
4. 通过第一步给定的$x,y$计算出新的$z^n$
5. 将新的$(x^n,y^n,z^n)$变成初始猜测值
6. 重复2到5的过程，直到解的变化满足特定的精度要求则停止。

---
> * 方程中的上标`n,o`表示`new,old`，只是为了方便记忆而已。后续使用有所不同
> * 方程`o`表示旧，已知值，`n`表示新，未知待求解值
---

### Code

In [ ]:
"""
Jacobi
"""
import numpy as np

x_old = np.zeros((3, 1))
x_new = np.zeros((3, 1))

print("-----------------------------------------------------")
print("Iteration    x             y              z")
print("-----------------------------------------------------")
for k in range(100):
    x_new[0] = ( 9 - 2*x_old[1] - 2*x_old[2]) / 5
    x_new[1] = ( 1 + 2*x_old[0] + 3*x_old[2]) / 6
    x_new[2] = (10 -   x_old[0] - 2*x_old[1]) / 7
    print("%5d       |%f|    |%f|     |%f|" %(k+1, x_new[0], x_new[1], x_new[2]))
    if (np.linalg.norm(x_new - x_old) < 1e-6):
        print("-----------------------------------------------------")
        break
    if (k+1) % 5 == 0:
        print("-----------------------------------------------------")
    x_old = np.copy(x_new)

> 1. 收敛误差是精确数值解与部分迭代解之间的差值
> 2. 不停迭代直到收敛误差达到机器精度
> 3. 当解随着迭代过程偏离正确值，即为`发散`

`影响收敛的两个因素`
1. 系数矩阵性质
2. 迭代解法

将上述例题1, 第三行改成$7x+2y+z=10$

### Code

In [ ]:
"""
Jacobi
"""
import numpy as np

x_old = np.zeros((3, 1))
x_new = np.zeros((3, 1))

print("-----------------------------------------------------")
print("Iteration    x             y              z")
print("-----------------------------------------------------")
for k in range(100):
    x_new[0] = ( 9 - 2*x_old[1] - 2*x_old[2]) / 5
    x_new[1] = ( 1 + 2*x_old[0] + 3*x_old[2]) / 6
    x_new[2] = (10 - 1*x_old[0] - 2*x_old[1]) / 7
    print("%5d       |%f|    |%f|     |%f|" %(k+1, x_new[0], x_new[1], x_new[2]))
    if (np.linalg.norm(x_new - x_old) < 1e-6):
        print("-----------------------------------------------------")
        print("iteration number is: ", k+1)
        break
    if (k+1) % 5 == 0:
        print("-----------------------------------------------------")
    x_old = np.copy(x_new)

通过观察可以发现，可以发现两个矩阵性质的不同，可以发现第一个矩阵具备下面性质

$$
a_{i,i}\ge\sum_{\substack{j=1\\j\neq i}}|a_{i,j}|
$$

符合这个性质的矩阵，我们称其为`对角占优`矩阵，又称其为[Scarborough criterion](https://en.wikipedia.org/wiki/Scarborough_criterion)

<font color=red>注意</font>
1. 并不是说，不满足对角占优条件的矩阵就不收敛
2. 满足对角占优的一定收敛

## 方程的残差和修正形式

以[Lecture11 对流扩散问题有限体积法三](https://q8frym1nsp.feishu.cn/docx/IkKDdGZ1Yo4Q5ZxSYbWc3fgunvc)为例。有限体积法离散后

局部编号下通用形式为
$$
\tag{4}
a_P\phi_P+\sum_{nb}a_{nb}\phi_{nb}=S_u
$$

整体编号下通用形式为

$$
\tag{5}
a_{i,j}\phi_{i,j}+a_{i+1,j}\phi_{i+1,j}+a_{i-1,j}\phi_{i-1,j}+a_{i,j+1}\phi_{i,j+1}
+a_{i,j-1}\phi_{i,j-1}=S_u
$$

具体系数，想必大家已经清楚，不在赘述。

令$$k=i+(j-1)*N$$，则公式5可写为

$$
\tag{6}
a_k\phi_k+a_{k+1}\phi_{k+1}+a_{k-1}\phi_{k-1}+a_{k+N}\phi_{k+N}+a_{k-N}\phi_{k-N}=S_k
$$

通常我们说这是`五点格式`

如果我们通过迭代法，求解方程6，经过$n$次迭代可得$\phi^n$，但是将其代回方程$6$，左右两边一定相等。如果相等则收敛。则

$$
\tag{7}
R_k^n=S_k-a_k\phi_k^n-a_{k+1}\phi_{k+1}^n-a_{k-1}\phi_{k-1}^n-a_{k+N}\phi_{k+N}^n-a_{k-N}\phi_{k-N}^n
$$

---

<font color=red>注意</font>

1. $R_k^n$成为残差向量，列向量，用来判断是否收敛
2. 如果残差向量的分量都为$0$，则收敛
---

假设迭代$n+1$步，物理量$\phi$在第n步和第n+1步之间的关系表示为，$\phi^{n+1}=\phi^n+\phi'$，则根据公式$6$可以得到下式

$$
\tag{8}
a_k(\phi_k^{n}+\phi'_k)+a_{k+1}(\phi_{k+1}^{n}+\phi'_{k+1})+\\
a_{k-1}(\phi_{k-1}^{n}+\phi'_{k-1})+a_{k+N}(\phi_{k+N}^{n}+\phi'_{k+N})+a_{k-N}(\phi_{k-N}^{n}+\phi'_{k-N})\\=S_k
$$

因为第n次迭代的$\phi^n$为已知值，将方程8改写整理一下，可得

$$\tag{9}
\boxed{a_k\phi'_k+\sum_{nb}a_{nb}\phi'_{nb}=R_k^n}\\=S_k-a_k\phi_k^n-a_{k+1}\phi_{k+1}^n-a_{k-1}\phi_{k-1}^n-a_{k+N}\phi_{k+N}^n-a_{k-N}\phi_{k-N}^n
$$

方程$9$就是线性方程组的修正形式。相对于方程$5$，我们更喜欢方程$9$的形式。因为收敛的时候，方程5是左右相等，而方程9是左右两边都为零。

使用两条标准来判别收敛

$$
\tag{10a}
R_1=\sum_{k=1}^k|R_k|
$$

$$
\tag{10b}
R_2=\sqrt{\sum_{k=1}^k(R_k)^2}
$$

这就是所谓的1和2范数。

<font color=red>注意</font>   
1. 残差使用1范数时，因为残差向量中的分量有正、有负，如果不加绝对值可能带来意想不到的效果！
2. 10b的计算非常简单，可以用两个向量的内积来计算，$R_2=\sqrt{R'\cdot R}$

### 收敛标准

$$\tag{11}
R_2 < \epsilon_{tolerance}
$$

残差标准是人为根据不同问题设定的。


## Jacobi Method

最开始例题实际上就是使用的雅可比迭代法，其公式为

$$
\tag{12}
\phi_k^{\color{blue}{n+1}}=\dfrac{S_k-\sum_{nb}a_{nb}\phi_{nb}^{\color{red}n}}{a_k}
$$

![雅可比](./images/1_jacobi.png "雅可比")

图片中绿色方块表示显式处理，蓝色圆圈表示隐式处理

雅可比方法又称为`逐点迭代方法(point-wise iteration)`

## Gauss-Seidel Method

以五点格式为例，其公式如下

$$
\tag{13}
\phi_k^{\color{blue}{n+1}}=\dfrac{S_k-a_{k+1}\phi_{k+1}^{\color{red}n}-a_{k-1}\phi_{k-1}^{\color{blue}{n+1}}-a_{k+N}\phi_{k+N}^{\color{red}n}-a_{k-N}\phi_{k-N}^{\color{blue}{n+1}}}{a_k}
$$

![gauss_seidel](./images/2_gauss.png "高斯塞德尔")

图片中绿色方块表示显式处理，蓝色圆圈表示隐式处理

高斯塞德尔方法的显式元素别雅可比多，所以计算速度就块一些。

### code

In [ ]:
"""
Gauss
"""
import numpy as np

x_old = np.zeros((3, 1))
x_new = np.zeros((3, 1))

print("-----------------------------------------------------")
print("Iteration    x             y              z")
print("-----------------------------------------------------")
for k in range(100):
    x_new[0] = ( 9 - 2*x_old[1] - 2*x_old[2]) / 5
    x_new[1] = ( 1 + 2*x_new[0] + 3*x_old[2]) / 6
    x_new[2] = (10 - 1*x_new[0] - 2*x_new[1]) / 7
    print("%5d       |%f|    |%f|     |%f|" %(k+1, x_new[0], x_new[1], x_new[2]))
    if (np.linalg.norm(x_new - x_old) < 1e-6):
        print("-----------------------------------------------------")
        print("iteration number is: ", k+1)
        break
    if (k+1) % 5 == 0:
        print("-----------------------------------------------------")
    x_old = np.copy(x_new)

### Row-wise and Column-wise



以五点格式为例

![row_column_wise](./images/3_row.png "行列扫描")

其计算公式为

$$
\tag{13}
a_k\phi_k^{\color{blue}{n+1}}+a_{k-1}\phi_{k-1}^{\color{blue}{n+1}}+a_{k+1}\phi_{k+1}^{\color{blue}{n+1}}=S_k+a_{k+N}\phi_{k+N}^{\color{red}{n}}+a_{k-N}\phi_{k-N}^{\color{blue}{n+1}}
$$

<font color=red>注意</font>
1. 细细体会图片和公式
2. 这里要进行一次矩阵计算